<a href="https://colab.research.google.com/github/stepsbtw/Algoritmos/blob/main/algorithms_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Leitura e Filtragem inicial dos dados
Carregue os dados em sua linguagem de programação preferida (por exemplo, Python com pandas).

In [3]:
import pandas as pd
import requests
from io import BytesIO

r = requests.get("https://data.brasil.io/dataset/covid19/caso.csv.gz")
caso = pd.read_csv(BytesIO(r.content), compression="gzip")

r = requests.get("https://data.brasil.io/dataset/covid19/caso_full.csv.gz")
caso_full = pd.read_csv(BytesIO(r.content), compression="gzip")

In [4]:
caso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838003 entries, 0 to 2838002
Data columns (total 13 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   date                            object 
 1   state                           object 
 2   city                            object 
 3   place_type                      object 
 4   confirmed                       int64  
 5   deaths                          int64  
 6   order_for_place                 int64  
 7   is_last                         bool   
 8   estimated_population_2019       float64
 9   estimated_population            float64
 10  city_ibge_code                  float64
 11  confirmed_per_100k_inhabitants  float64
 12  death_rate                      float64
dtypes: bool(1), float64(5), int64(3), object(4)
memory usage: 262.5+ MB


In [5]:
caso_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853648 entries, 0 to 3853647
Data columns (total 18 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   city                                           object 
 1   city_ibge_code                                 float64
 2   date                                           object 
 3   epidemiological_week                           int64  
 4   estimated_population                           float64
 5   estimated_population_2019                      float64
 6   is_last                                        bool   
 7   is_repeated                                    bool   
 8   last_available_confirmed                       int64  
 9   last_available_confirmed_per_100k_inhabitants  float64
 10  last_available_date                            object 
 11  last_available_death_rate                      float64
 12  last_available_deaths                     

Descrição das colunas:

- **city: nome do município (pode estar em branco quando o registro é referente ao estado, pode ser preenchido com Importados/Indefinidos também).**
- city_ibge_code: código IBGE do local.
- **date: data de coleta dos dados no formato YYYY-MM-DD.**
- epidemiological_week: número da semana epidemiológica no formato YYYYWW.
- estimated_population: população estimada para esse município/estado em 2020, segundo o IBGE. (acesse o script que faz o download e conversão dos dados de população).
- estimated_population_2019: população estimada para esse município/estado em 2019, segundo o IBGE. ATENÇÃO: essa coluna possui valores desatualizados, prefira usar a coluna estimated_population.
- is_last: campo pré-computado que diz se esse registro é o mais novo para esse local, pode ser True ou False (caso filtre por esse campo, use is_last=True ou is_last=False, não use o valor em minúsculas).
- is_repeated: campo pré-computado que diz se as informações nesse registro foram publicadas pela Secretaria Estadual de Saúde no dia date ou se o dado é repetido do último dia em que o dado está disponível (igual ou anterior a date). Isso ocorre pois nem todas as secretarias publicam boletins todos os dias. Veja também o campo last_available_date.
- **last_available_confirmed: número de casos confirmados do último dia disponível igual ou anterior à data date.**
- **last_available_confirmed_per_100k_inhabitants: número de casos confirmados por 100.000 habitantes (baseado em estimated_population) do último dia disponível igual ou anterior à data date.**
- last_available_date: data da qual o dado se refere.
- **last_available_death_rate: taxa de mortalidade (mortes / confirmados) do último dia disponível igual ou anterior à data date.**
- **last_available_deaths: número de mortes do último dia disponível igual ou anterior à data date.**
- order_for_place: número que identifica a ordem do registro para este local. O registro referente ao primeiro boletim em que esse local aparecer será contabilizado como 1 e os demais boletins incrementarão esse valor.
- place_type: tipo de local que esse registro descreve, pode ser city ou state.
- **state: sigla da unidade federativa, exemplo: SP.**
- new_confirmed: número de novos casos confirmados desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).
- new_deaths: número de novos óbitos desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).

Recomendadas para análise:
- city
- state
- last_avaible_confirmed
- last_available_deaths
- estimated_population
- last_available_confirmed_per_100k_inhabitants
- last_available_death_rate
- date

Para análises estatísticas mais simples:

In [6]:
# Uma linha por município
df = caso[(caso["is_last"]==True) & (caso["place_type"]=="city")]

Para análise de desempenho de algoritmos com volumes maiores de dados (várias datas por município):

In [7]:
# Séries temporais completas.
df_full = caso_full[(caso_full["place_type"]=="city") & (caso_full["is_repeated"]==False)]

Checar e tratar (remover linhas) colunas com valores nulos.

In [8]:
df.count()

,0
date,5589
state,5589
city,5589
place_type,5589
confirmed,5589
deaths,5589
order_for_place,5589
is_last,5589
estimated_population_2019,5570
estimated_population,5570


In [9]:
df_clean = df.dropna(subset=["confirmed_per_100k_inhabitants", "estimated_population"])
df_full.count()

,0
city,2818241
city_ibge_code,2807856
date,2818241
epidemiological_week,2818241
estimated_population,2807856
estimated_population_2019,2807856
is_last,2818241
is_repeated,2818241
last_available_confirmed,2818241
last_available_confirmed_per_100k_inhabitants,2793772


In [10]:
df_full_clean = df_full.dropna(subset=["last_available_confirmed_per_100k_inhabitants", "estimated_population"])

## 2. Cálculo de Métricas
Para cada município:
- Taxa de casos por 100 mil habitantes:
  - casos/habitantes * 100 000
- Taxa de óbitos por 100 mil habitantes:
  - obitos/habitantes * 100 000
- Taxa de mortalidade:
  - obitos/casos

In [11]:
# Atenção! Existem muitos municípios com o mesmo nome em estados diferentes:
counts = df_clean["city"].value_counts()
counts[counts>1]

,count
city,
São Domingos,5
Bom Jesus,5
Planalto,4
São Francisco,4
Vera Cruz,4
...,...
Petrolândia,2
Tabatinga,2
São Carlos,2


Portanto vamos usar o indentificador único do ibge.

In [12]:
df_resposta = df_clean[["city", "state", "city_ibge_code"]].copy()

df_resposta["taxa_casos_100k"] = df_clean["confirmed"] / df_clean["estimated_population"] * 10**5
df_resposta["taxa_obitos_100k"] = df_clean["deaths"] / df_clean["estimated_population"] * 10**5
df_resposta["taxa_mortalidade"] = df_clean["deaths"] / df_clean["confirmed"]

df_resposta

,city,state,city_ibge_code,taxa_casos_100k,taxa_obitos_100k,taxa_mortalidade
734,Amapá,AP,1600105.0,15500.163274,141.504300,0.009129
1320,Calçoene,AP,1600204.0,16380.682823,123.828056,0.007559
1901,Cutias,AP,1600212.0,14325.520406,114.735289,0.008009
2478,Ferreira Gomes,AP,1600238.0,19656.081336,87.862433,0.004470
3058,Itaubal,AP,1600253.0,9791.703756,53.409293,0.005455
...,...,...,...,...,...,...
2835558,Águas de São Pedro,SP,3500600.0,13092.871343,340.812269,0.026030
2836080,Álvares Florence,SP,3501202.0,17164.792981,438.716753,0.025559
2836553,Álvares Machado,SP,3501301.0,10028.802304,316.025282,0.031512
2837057,Álvaro de Carvalho,SP,3501400.0,5536.594615,170.648464,0.030822


## 3. Ordenação:
Implemente os algoritmos de ordenação abaixo e aplique-os para classificar os municípios por:
- Maior taxa de casos por 100 mil hab.
- Maior taxa de mortalidade.
- Menor taxa de mortalidade.

Compare:
- Merge Sort
- Quick Sort
- Função nativa da linguagem (ex: sort() do Python)

In [13]:
def merge(vL, vR, compare): # auxiliary O(n) space
  out = []
  i = j = 0
  # inserting
  while i < len(vL) and j < len(vR):
    if compare(vL[i], vR[j]): # rev
      out.append(vL[i])
      i += 1
    else:
      out.append(vR[j])
      j += 1
  # extending
  while i < len(vL):
    out.append(vL[i])
    i += 1
  while j < len(vR):
    out.append(vR[j])
    j += 1

  return out

def mergesort(v, l=0, r=None, compare=lambda x,y : x<y):
  if r == None:
    r = len(v)-1
  # base case
  if l == r:
    return [v[l]]
  # divide T(n//2)
  m = (l+r)//2
  vL = mergesort(v, l, m, compare)
  vR = mergesort(v, m+1, r, compare)
  # conquer O(n)
  return merge(vL, vR, compare)

In [14]:
def partition(v, l, r, compare):
  pivo = v[r] # random
  i = l
  for j in range(l,r):
    if compare(v[j], pivo):
      v[i], v[j] = v[j], v[i]
      i+=1
  v[i], v[r] = v[r], v[i]
  return i

def quicksort(v, l=0, r=None, compare=lambda x,y : x<y):  # inplace
  if r == None:
    r = len(v)-1
  if l < r:
    # divide (random)
    pidx = partition(v, l, r, compare)
    # conquer
    quicksort(v, l, pidx-1, compare)
    quicksort(v, pidx+1, r, compare)

Devido ao grande tamanho da base de dados, seria mais interessante implementar as versões iterativas dos algoritmos para evitar estourar a pilha de recursão.

In [15]:
def merge(v, l1, r1, l2, r2, compare):
  out = []
  i, j = l1, l2
  while i <= r1 and j <= r2:
    if compare(v[i], v[j]):
      out.append(v[i])
      i += 1
    else:
      out.append(v[j])
      j += 1

  while i <= r1:
    out.append(v[i])
    i += 1
  while j <= r2:
    out.append(v[j])
    j += 1
  return out

def mergesort(v, l=0, r=None, compare=lambda x,y : x<y):
  if r == None:
    r = len(v)-1
  n = len(v)
  if n <= 1:
    return v
  stack = [(0, n-1, False)]
  while stack:
    l, r, divided = stack.pop()
    if l < r:
      m = (l + r) // 2
      if not divided:
        # dividir
        stack.append((l, r, True)) # marcar como dividido
        stack.append((m+1, r, 0)) # direita
        stack.append((l, m, 0)) # esquerda
      else:
        # intercalar [l:m] e [m+1:r]
        merge(v, l, m, m+1, r, compare)

In [16]:
def partition(v, l, r, compare):
  pivo = v[r] # random
  i = l
  for j in range(l,r):
    if compare(v[j], pivo):
      v[i], v[j] = v[j], v[i]
      i+=1
  v[i], v[r] = v[r], v[i]
  return i

def quicksort(v, l=0, r=None, compare=lambda x,y : x<y):
  if r == None:
    r = len(v)-1
  stack = [(l,r)]
  while stack:
    l,r = stack.pop()
    if l < r:
      pidx = partition(v, l, r, compare)
      stack.append((l,pidx-1))
      stack.append((pidx+1,r))

In [17]:
def merge(v, l1, r1, l2, r2, compare):
  out = []
  i, j = l1, l2

  while i <= r1 and j <= r2:
    if compare(v[i], v[j]):
      out.append(v[i])
      i += 1
    else:
      out.append(v[j])
      j += 1

  while i <= r1:
    out.append(v[i])
    i += 1
  while j <= r2:
    out.append(v[j])
    j += 1

  for k in range(len(out)):
        v[l1 + k] = out[k]

def mergesort(v, compare=lambda x,y : x<y):
  n = len(v)
  tam = 1 # tamanho das sublistas

  while tam < n:
    for i in range(0, n, 2*tam):
      l1 = i
      r1 = min(i + tam-1, n-1)
      l2 = r1+1
      r2 = min(i + 2*tam-1, n-1)

      if l2 <= r2:
        merge(v, l1, r1, l2, r2, compare)
    tam *= 2

  return v

Aplicação:

import time
Para ordenar os DataFrames apartir das colunas, precisaria salvar um array de índices ordenados.

In [51]:
import time

def make_idx(v):
  return list(range(len(v)))

def sort_benchmark(df, column, sample_size, reverse=False):
    data = df.sample(sample_size)[column].reset_index(drop=True)
    idx_m, idx_q, idx_s = make_idx(data), make_idx(data), make_idx(data)

    # Choose comparator based on sort direction
    cmp = (lambda i, j: data[i] >= data[j]) if reverse else (lambda i, j: data[i] <= data[j])

    # Mergesort
    start = time.time()
    mergesort(idx_m, compare=cmp)
    end = time.time()
    print(f"Mergesort: {end - start} segundos")

    # Quicksort
    start = time.time()
    quicksort(idx_q, compare=cmp)
    end = time.time()
    print(f"Quicksort: {end - start} segundos")

    # Timsort
    start = time.time()
    idx_s.sort(key=lambda i: data[i], reverse=reverse)
    end = time.time()
    print(f"Timsort: {end - start} segundos\n")


In [52]:
sort_benchmark(df_full_clean, "last_available_confirmed_per_100k_inhabitants", 100, reverse=True)
sort_benchmark(df_full_clean, "last_available_death_rate", 100, reverse=True)
sort_benchmark(df_full_clean, "last_available_death_rate", 100)

Mergesort: 0.0022895336151123047 segundos
Quicksort: 0.002428770065307617 segundos
Timsort: 0.00022363662719726562 segundos

Mergesort: 0.002249002456665039 segundos
Quicksort: 0.0028743743896484375 segundos
Timsort: 0.0002353191375732422 segundos

Mergesort: 0.0022020339965820312 segundos
Quicksort: 0.0024869441986083984 segundos
Timsort: 0.0002014636993408203 segundos



In [53]:
sort_benchmark(df_full_clean, "last_available_confirmed_per_100k_inhabitants", 1000, reverse=True)
sort_benchmark(df_full_clean, "last_available_death_rate", 1000, reverse=True)
sort_benchmark(df_full_clean, "last_available_death_rate", 1000)

Mergesort: 0.03561997413635254 segundos
Quicksort: 0.04230785369873047 segundos
Timsort: 0.0021295547485351562 segundos

Mergesort: 0.03285551071166992 segundos
Quicksort: 0.073944091796875 segundos
Timsort: 0.002626180648803711 segundos

Mergesort: 0.03384709358215332 segundos
Quicksort: 0.07705998420715332 segundos
Timsort: 0.0020835399627685547 segundos



In [54]:
sort_benchmark(df_full_clean, "last_available_confirmed_per_100k_inhabitants", 10000, reverse=True)
sort_benchmark(df_full_clean, "last_available_death_rate", 10000, reverse=True)
sort_benchmark(df_full_clean, "last_available_death_rate", 10000)

Mergesort: 0.7797179222106934 segundos
Quicksort: 0.8382136821746826 segundos
Timsort: 0.022511005401611328 segundos

Mergesort: 0.8892745971679688 segundos
Quicksort: 5.507475852966309 segundos
Timsort: 0.03126025199890137 segundos

Mergesort: 0.48665571212768555 segundos
Quicksort: 4.786398649215698 segundos
Timsort: 0.02177119255065918 segundos



In [ ]:
sort_benchmark(df_full_clean, "last_available_confirmed_per_100k_inhabitants", 100000, reverse=True)
sort_benchmark(df_full_clean, "last_available_death_rate", 100000, reverse=True)
sort_benchmark(df_full_clean, "last_available_death_rate", 100000)

Mergesort: 6.751708745956421 segundos
Quicksort: 8.159952878952026 segundos
Timsort: 0.23743104934692383 segundos

Mergesort: 7.116086959838867 segundos


Quicksort é um algoritmo de ordenação INSTÁVEL, altera a ordenação de elementos iguais, já o Timsort (python nativo) e o Mergesort são estáveis.

O Quicksort também é INPLACE, altera o array original sem precisar de um espaço auxiliar. Já o Mergesort e o Timsort (python nativo) utilizam um espaço O(n) auxiliar.

Em complexidade de tempo, o Mergesort é O(nlogn) em todos os casos.

O Quicksort é um algoritmo aleatório, onde a escolha do pivô influência diretamente na complexidade. Seu melhor caso é quando os pivôs escolhidos dividem a lista no meio sucessivamente, sendo O(nlogn). No pior caso, onde a lista diminui em apenas 1 elemento, é O(n^2). É provado que em média o Quicksort é O(nlogn)

O Timsort é um método hibrido em que utiliza Mergesort e aproveita o Insertion sort para sublistas pequenas. O(nlogn) no pior e caso médio, O(n) no melhor (caso já esteja ordenado).

# 4) Busca da Mediana - Quickselect

- A mediana das taxas de mortalidade entre os municípios.
- A mediana das taxas de casos por 100 mil habitantes.

Compare o desempenho do Quickselect com a abordagem tradicional
de ordenar e acessar o elemento central.

# 5) Avaliação de desempenho
- Meça o tempo de execução de cada algoritmo para diferentes tamanho de entrada (e.g., 100, 1000, 10000 linhas).
- Compare os resultados com a complexidade esperada.


6) Relatório
- Gráficos de desempenho (tempo vs tamanho).
- Tabelas com os Top 10 municípios segundo cada critério.
- Valor da mediana das taxas encontradas e sua interpretação
- Análise crítica dos resultados (breve)